In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('datasets/parkinson_disease.csv')
pd.set_option('display.max_columns', 10)
df.sample(5)

,id,gender,PPE,DFA,RPDE,...,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
156,52,1,0.81255,0.60243,0.52083,...,2.5098,2.4328,2.7477,13.3490,1
323,107,1,0.80460,0.73701,0.32604,...,12.2715,12.1886,8.2745,41.8950,1
223,74,1,0.47019,0.75495,0.63768,...,16.8188,30.3167,27.3304,14.9176,1
232,77,0,0.63101,0.82411,0.80353,...,2.6998,3.0054,4.9437,4.8401,1
403,134,1,0.82159,0.67006,0.60602,...,59.3870,43.7500,38.0946,92.2488,1
